# Predikcia vsetkych parametrov pomocou jednej NN pre normovane detached data

In [1]:
# LIBRARIES
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
from keras.utils import np_utils
from keras.models import load_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import precision_recall_fscore_support
from keras.layers import Conv1D, GlobalMaxPooling1D, MaxPooling1D, SpatialDropout1D, GlobalAveragePooling1D
from keras.layers import Input, Dense, concatenate, Activation, LSTM, Dropout, Flatten
from keras.models import Model
from keras.layers.merge import Concatenate
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:
np.random.seed(1234)
pd.set_option('display.max_rows', None)

In [4]:
def generate_observation_sigma(space_obs_frac=0.5):
    """
    Draws a standard deviation of noise in light curve points from a "true" value provided in synthetic light curve.
    Noise sigma is drawn from bimodal distribution taking into account contributions from space based and earth based
    observations which have different levels of stochastic noise.

    :param space_obs_frac: ratio between earth based and space based observations
    :return: float; standard deviation of the light curve noise
    """
    earth_based_sigma = 4e-3
    space_based_sigma = 2e-4
    sigma = np.random.choice([earth_based_sigma, space_based_sigma], p=[1-space_obs_frac, space_obs_frac])
    return np.random.rayleigh(sigma)

def stochastic_noise_generator(curve):
    """
    Introduces gaussian noise into synthetic observation provided in `curve`.

    :param curve: numpy.array; normalized light curve
    :return: Tuple(numpy.array, float); normalized light curve with added noise, standard deviation of observations
    """
    sigma = generate_observation_sigma()
    return np.random.normal(curve, sigma), np.full(curve.shape, sigma)

### Data loading

In [5]:
data = pd.read_pickle("detached_all_parameters.pkl").reset_index()

In [6]:
data.head()

,index,id,curve,primary__t_eff,secondary__t_eff,inclination,mass_ratio,primary__surface_potential,secondary__surface_potential,t1_t2,filter,critical_surface_potential,primary__equivalent_radius,secondary__equivalent_radius,primary__filling_factor,secondary__filling_factor
0,0,38,"[0.6055271686415179, 0.9842041250556204, 0.999...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_U,15.09104,0.009996,0.009996,-145.333979,-1502.830354
1,1,38,"[0.608985656265516, 0.9846965713304289, 0.9998...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_B,15.09104,0.009996,0.009996,-145.333979,-1502.830354
2,2,38,"[0.6189025614226916, 0.9837351924934223, 0.999...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_V,15.09104,0.009996,0.009996,-145.333979,-1502.830354
3,3,38,"[0.6292771409565273, 0.9832675811171884, 0.999...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_R,15.09104,0.009996,0.009996,-145.333979,-1502.830354
4,4,38,"[0.6543378609145588, 0.9835188424579704, 0.999...",7000,4000,1.560796,10.0,110.00005,996.5005,1.75,Bessell_I,15.09104,0.009996,0.009996,-145.333979,-1502.830354


In [7]:
data_sample = data.sample(n=100000)

In [8]:
list(data_sample.columns)

['index',
 'id',
 'curve',
 'primary__t_eff',
 'secondary__t_eff',
 'inclination',
 'mass_ratio',
 'primary__surface_potential',
 'secondary__surface_potential',
 't1_t2',
 'filter',
 'critical_surface_potential',
 'primary__equivalent_radius',
 'secondary__equivalent_radius',
 'primary__filling_factor',
 'secondary__filling_factor']

### Train test split + Normalizacia y_train na interval <0,1> pomocou sklearn - MinMaxScaler

In [9]:
X = []
for row in data_sample["curve"]:
    X.append(row)
X=np.array(X)

In [10]:
y = np.array(data_sample[[
    "primary__t_eff",
    "secondary__t_eff",
    "inclination",
    "mass_ratio",
    "primary__surface_potential",
    "secondary__surface_potential",
    "t1_t2",
    "critical_surface_potential",
    "primary__equivalent_radius",
    "secondary__equivalent_radius",
    "primary__filling_factor",
    "secondary__filling_factor"]])

In [11]:
y[0]

array([ 4.50000000e+04,  5.00000000e+03,  1.33407634e+00,  6.00000000e-01,
        2.06012514e+01,  4.07107635e+00,  9.00000000e+00,  3.06344154e+00,
        4.99813455e-02,  2.10723804e-01, -4.99552049e+01, -2.87017614e+00])

In [12]:
X_train1, X_test, y_train1, y_test = train_test_split(X, y, test_size=0.2)

In [13]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
y_normed = scaler.fit_transform(y_train1)

In [14]:
y_normed[0]

array([0.14634146, 0.04878049, 0.52068157, 0.07070707, 0.01441232,
       0.00379881, 0.06504065, 0.1110169 , 0.66357441, 0.29285875,
       0.99976493, 0.99645338])

In [15]:
X_train = []
y_train = []
for i in range(len(X_train1)):
    for j in range(3):
        curve = stochastic_noise_generator(X_train1[i])
        X_train.append(curve[0])
        y_train.append(y_normed[i])
X_train = np.array(X_train)
y_train=np.array(y_train)

In [16]:
print("Number of records in dataset: ", len(data),
    "\nNumber of records in sample: ", len(X),
    "\nNumber of train data without noise: ", len(X_train1),
    "\nNumber of train data with noise: ", len(X_train),
    "\nNumber of test data without noise: ", len(X_test))

Number of records in dataset:  1300000 
Number of records in sample:  100000 
Number of train data without noise:  80000 
Number of train data with noise:  240000 
Number of test data without noise:  20000


## Model

In [17]:
inputs = Input(shape=(400, 1))
b = Conv1D(64, kernel_size = 3, padding = "valid")(inputs)
b = MaxPooling1D(2)(b)
b = Dropout(0.2)(b)
b = LSTM(64, return_sequences=True)(b)
b = Flatten()(b)
b = Dense(64, activation='relu')(b)
x = Dense(32, activation='relu')(b)
output = Dense(12, activation='linear')(x)
model = Model(inputs=inputs, outputs=output)
model.compile(loss='mse', optimizer='adam', metrics=["mae", "mape"])
print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 400, 1)]          0         
_________________________________________________________________
conv1d (Conv1D)              (None, 398, 64)           256       
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 199, 64)           0         
_________________________________________________________________
dropout (Dropout)            (None, 199, 64)           0         
_________________________________________________________________
lstm (LSTM)                  (None, 199, 64)           33024     
_________________________________________________________________
flatten (Flatten)            (None, 12736)             0         
_________________________________________________________________
dense (Dense)                (None, 64)               

In [18]:
saved_model = "models/norm_detached_allParams.hdf5"
checkpoint = ModelCheckpoint(saved_model, monitor = 'val_mae', verbose = 1, save_best_only = True, mode = 'min')
early = EarlyStopping(monitor = "val_mae", mode = "min", patience = 25)
callbacks_list = [checkpoint, early]

In [19]:
history = model.fit(X_train, y_train, validation_split = 0.1, epochs = 10, verbose = 1, callbacks = callbacks_list, batch_size = 64)

Epoch 1/10
3375/3375 [==============================] - ETA: 0s - loss: 0.0264 - mae: 0.1042 - mape: 4934485.5000
Epoch 00001: val_mae improved from inf to 0.09018, saving model to models\norm_detached_allParams.hdf5
3375/3375 [==============================] - 648s 192ms/step - loss: 0.0264 - mae: 0.1042 - mape: 4934485.5000 - val_loss: 0.0216 - val_mae: 0.0902 - val_mape: 3767850.0000
Epoch 2/10
3375/3375 [==============================] - ETA: 0s - loss: 0.0212 - mae: 0.0903 - mape: 3834508.0000
Epoch 00002: val_mae improved from 0.09018 to 0.08416, saving model to models\norm_detached_allParams.hdf5
3375/3375 [==============================] - 637s 189ms/step - loss: 0.0212 - mae: 0.0903 - mape: 3834508.0000 - val_loss: 0.0199 - val_mae: 0.0842 - val_mape: 3556810.0000
Epoch 3/10
3375/3375 [==============================] - ETA: 0s - loss: 0.0196 - mae: 0.0855 - mape: 3616911.7500
Epoch 00003: val_mae improved from 0.08416 to 0.08067, saving model to models\norm_detached_allParams.

In [17]:
model = load_model("models/norm_detached_allParams.hdf5")

### Model evaluation on normalized test data

In [18]:
y_test_normed = scaler.fit_transform(y_test)

In [19]:
# Evaluated on normalized test data without noise
model.evaluate(X_test, y_test_normed)

625/625 [==============================] - 33s 53ms/step - loss: 0.0147 - mean_absolute_error: 0.0687 - mean_absolute_percentage_error: 2674469.5000


[0.014678007923066616, 0.06868328154087067, 2674469.5]

In [19]:
# Adding noise to normalized test data
X_test_n = []
y_test_norm_n = []
for i in range(len(X_test)):
    for j in range(3):
        curve = stochastic_noise_generator(X_test[i])
        X_test_n.append(curve[0])
        y_test_norm_n.append(y_test_normed[i])
        j += 1

In [20]:
X_test_n = np.array(X_test_n)
y_test_norm_n = np.array(y_test_norm_n)

In [22]:
# EVALUATED ON NORMALIZED TEST DATA WITH NOISE
model.evaluate(X_test_n, y_test_norm_n)

1875/1875 [==============================] - 123s 66ms/step - loss: 0.0152 - mean_absolute_error: 0.0701 - mean_absolute_percentage_error: 2786056.0000


[0.015210004523396492, 0.07005462050437927, 2786056.0]

## Prediction on normalized test data without noise

### working with normalized prediction and test data

In [21]:
y_pred_norm = model.predict(X_test)

In [22]:
P_prim__t_eff = [i[0] for i in y_pred_norm]
P_sec__t_eff = [i[1] for i in y_pred_norm]
P_inclination = [i[2] for i in y_pred_norm]
P_mass_ratio = [i[3] for i in y_pred_norm]
P_prim__surface_potential = [i[4] for i in y_pred_norm]
P_sec__surface_potential = [i[5] for i in y_pred_norm]
P_t1_t2 = [i[6] for i in y_pred_norm]
P_critical_surface_potential = [i[7] for i in y_pred_norm]
P_primary_equivalent_radius = [i[8] for i in y_pred_norm]
P_secondary_equivalent_radius = [i[9] for i in y_pred_norm]
P_primary_filling_factor = [i[10] for i in y_pred_norm]
P_secondary_filling_factor = [i[11] for i in y_pred_norm]

In [23]:
P_prim__t_eff = np.reshape(P_prim__t_eff, len(P_prim__t_eff))
P_sec__t_eff = np.reshape(P_sec__t_eff, len(P_sec__t_eff))
P_inclination = np.reshape(P_inclination, len(P_inclination))
P_mass_ratio = np.reshape(P_mass_ratio, len(P_mass_ratio))
P_prim__surface_potential = np.reshape(P_prim__surface_potential, len(P_prim__surface_potential))
P_sec__surface_potential = np.reshape(P_sec__surface_potential, len(P_sec__surface_potential))
P_t1_t2 = np.reshape(P_t1_t2, len(P_t1_t2))
P_critical_surface_potential = np.reshape(P_critical_surface_potential, len(P_critical_surface_potential))
P_primary_equivalent_radius = np.reshape(P_primary_equivalent_radius, len(P_primary_equivalent_radius))
P_secondary_equivalent_radius = np.reshape(P_secondary_equivalent_radius, len(P_secondary_equivalent_radius))
P_primary_filling_factor = np.reshape(P_primary_filling_factor, len(P_primary_filling_factor))
P_secondary_filling_factor = np.reshape(P_secondary_filling_factor, len(P_secondary_filling_factor))


In [24]:
df = pd.DataFrame(data =
    {
    'primary__t_eff': [i[0] for i in y_test_normed],
    'P_prim__t_eff': P_prim__t_eff,
    'prim_t_eff_abs_diff': abs([i[0] for i in y_test_normed] - P_prim__t_eff),

    'secondary__t_eff': [i[1] for i in y_test_normed],
    'P_sec__t_eff': P_sec__t_eff,
    'sec_t_eff_abs_diff': abs([i[1] for i in y_test_normed] - P_sec__t_eff),

    'inclination': [i[2] for i in y_test_normed],
    'P_inclination': P_inclination,
    'inclination_abs_diff': abs([i[2] for i in y_test_normed] - P_inclination),

    'mass_ratio': [i[3] for i in y_test_normed],
    'P_mass_ratio': P_mass_ratio,
    'mass_ratio_abs_diff': abs([i[3] for i in y_test_normed] - P_mass_ratio),

    'prim_surface_potential': [i[4] for i in y_test_normed],
    'P_prim_surf_potential': P_prim__surface_potential,
    'prim_surf_potent_abs_diff': abs([i[4] for i in y_test_normed] - P_prim__surface_potential),

    'sec_surface_potential': [i[5] for i in y_test_normed],
    'P_sec_surf_potential': P_sec__surface_potential,
    'sec_surf_potent_abs_diff': abs([i[5] for i in y_test_normed] - P_sec__surface_potential),

    't1_t2': [i[6] for i in y_test_normed],
    'P_t1_t2': P_t1_t2,
    't1_t2_abs_diff': abs([i[6] for i in y_test_normed] - P_t1_t2),

    'critical_surf_potential': [i[7] for i in y_test_normed],
    'P_crit_surf_potential': P_critical_surface_potential,
    'crit_surf_potential_abs_diff': abs([i[7] for i in y_test_normed] - P_critical_surface_potential),

    'prim_equi_radius': [i[8] for i in y_test_normed],
    'P_prim_equi_radius': P_primary_equivalent_radius,
    'prim_equi_radius_abs_diff': abs([i[8] for i in y_test_normed] - P_primary_equivalent_radius),

    'sec_equi_radius': [i[9] for i in y_test_normed],
    'P_sec_equi_radius': P_secondary_equivalent_radius,
    'sec_equi_radius_abs_diff': abs([i[9] for i in y_test_normed] - P_secondary_equivalent_radius),

    'prim_filling_factor': [i[10] for i in y_test_normed],
    'P_prim_filling_factor': P_primary_filling_factor,
    'prim_filling_factor_abs_diff': abs([i[10] for i in y_test_normed] - P_primary_filling_factor),

    'sec_filling_factor': [i[11] for i in y_test_normed],
    'P_sec_filling_factor': P_secondary_filling_factor,
    'sec_filling_factor_abs_diff': abs([i[11] for i in y_test_normed] - P_secondary_filling_factor)
    })


In [25]:
df.head()

,primary__t_eff,P_prim__t_eff,prim_t_eff_abs_diff,secondary__t_eff,P_sec__t_eff,sec_t_eff_abs_diff,inclination,P_inclination,inclination_abs_diff,mass_ratio,...,prim_equi_radius_abs_diff,sec_equi_radius,P_sec_equi_radius,sec_equi_radius_abs_diff,prim_filling_factor,P_prim_filling_factor,prim_filling_factor_abs_diff,sec_filling_factor,P_sec_filling_factor,sec_filling_factor_abs_diff
0,0.756098,0.453428,0.302670,0.634146,0.314589,0.319558,0.791784,0.801855,0.010071,0.242424,...,0.155613,0.438463,0.285876,0.152588,0.991633,0.993019,0.001386,0.995217,0.992953,0.002264
1,0.048780,0.431722,0.382942,0.024390,0.164085,0.139695,0.715846,0.825503,0.109656,1.000000,...,0.079097,0.365116,0.129231,0.235885,0.984807,0.967345,0.017463,0.970197,0.963762,0.006435
2,0.341463,0.418484,0.077021,0.146341,0.219176,0.072835,0.863052,0.853649,0.009403,0.080808,...,0.114110,0.366339,0.433634,0.067295,0.999461,0.995676,0.003786,0.997632,0.995528,0.002104
3,0.756098,0.515680,0.240418,0.146341,0.113871,0.032470,0.839451,0.811900,0.027551,0.070707,...,0.136597,0.440672,0.545582,0.104911,0.999765,1.000110,0.000345,0.998657,0.998345,0.000312
4,0.756098,0.583420,0.172677,0.097561,0.071386,0.026175,0.813617,0.811826,0.001791,0.090909,...,0.079931,0.146257,0.222856,0.076599,0.993935,0.990310,0.003626,0.989753,0.994659,0.004906


In [29]:
df.to_csv('all_params_predictions/DetachedSynthetic_normalized_allParams_OneModel.csv', index=False)

In [26]:
# Dictionary with averages of absolute differencies between predicted and true data
avg_diff_dict = {
    "avg_primary_t_eff": (sum(df.prim_t_eff_abs_diff)/len(df.prim_t_eff_abs_diff)),
    "avg_secondary_prim_t_eff": (sum(df.sec_t_eff_abs_diff)/len(df.sec_t_eff_abs_diff)),
    "avg_inclination": (sum(df.inclination_abs_diff)/len(df.inclination_abs_diff)),
    "avg_mass_ratio": (sum(df.mass_ratio_abs_diff)/len(df.mass_ratio_abs_diff)),
    "avg_primary_surface_potential":(sum(df.prim_surf_potent_abs_diff)/len(df.prim_surf_potent_abs_diff)),
    "avg_secondary_surface_potential": (sum(df.sec_surf_potent_abs_diff)/len(df.sec_surf_potent_abs_diff)),
    "avg_t1_t2":(sum(df.t1_t2_abs_diff)/len(df.t1_t2_abs_diff)),
    "avg_crit_surf_potent":(sum(df.crit_surf_potential_abs_diff)/len(df.crit_surf_potential_abs_diff)),
    "avg_prim_equi_radius":(sum(df.prim_equi_radius_abs_diff)/len(df.prim_equi_radius_abs_diff)),
    "avg_sec_equi_radius":(sum(df.sec_equi_radius_abs_diff)/len(df.sec_equi_radius_abs_diff)),
    "avg_prim_fill_factor":(sum(df.prim_filling_factor_abs_diff)/len(df.prim_filling_factor_abs_diff)),
    "avg_sec_fill_factor":(sum(df.sec_filling_factor_abs_diff)/len(df.sec_filling_factor_abs_diff))
}

In [27]:
# SCALED PREDICTED AND TEST VALUES
pd.DataFrame(avg_diff_dict.items(), columns = ['Attribute', 'Avg diff'])

,Attribute,Avg diff
0,avg_primary_t_eff,0.180490
1,avg_secondary_prim_t_eff,0.086314
2,avg_inclination,0.061597
3,avg_mass_ratio,0.105724
4,avg_primary_surface_potential,0.052827
5,avg_secondary_surface_potential,0.015211
6,avg_t1_t2,0.062996
7,avg_crit_surf_potent,0.108016
8,avg_prim_equi_radius,0.055528
9,avg_sec_equi_radius,0.065720


## Normalized predicted values andd test values to original

In [28]:
unscaled_test = scaler.inverse_transform(y_test_normed)
unscaled_test[0]

array([ 3.50000000e+04,  3.00000000e+04,  1.39733788e+00,  2.50000000e+00,
        1.36152222e+01,  1.03295175e+01,  1.16666667e+00,  5.94523973e+00,
        9.00097983e-02,  2.49770299e-01, -1.25745279e+01, -7.18778997e+00])

In [29]:
unscaled_pred = scaler.inverse_transform(y_pred_norm)
unscaled_pred[0]

array([ 2.2590541e+04,  1.6898137e+04,  1.4057275e+00,  1.4319487e+00,
        7.5857635e+00,  1.0101730e+01,  1.0946364e+00,  4.2177558e+00,
        1.7510697e-01,  1.6632760e-01, -1.0491901e+01, -1.0590524e+01],
      dtype=float32)

In [30]:
u_P_prim__t_eff = [i[0] for i in unscaled_pred]
u_P_sec__t_eff = [i[1] for i in unscaled_pred]
u_P_inclination = [i[2] for i in unscaled_pred]
u_P_mass_ratio = [i[3] for i in unscaled_pred]
u_P_prim__surface_potential = [i[4] for i in unscaled_pred]
u_P_sec__surface_potential = [i[5] for i in unscaled_pred]
u_P_t1_t2 = [i[6] for i in unscaled_pred]
u_P_critical_surface_potential = [i[7] for i in unscaled_pred]
u_P_primary_equivalent_radius = [i[8] for i in unscaled_pred]
u_P_secondary_equivalent_radius = [i[9] for i in unscaled_pred]
u_P_primary_filling_factor = [i[10] for i in unscaled_pred]
u_P_secondary_filling_factor = [i[11] for i in unscaled_pred]

In [31]:
u_P_prim__t_eff = np.reshape(u_P_prim__t_eff, len(u_P_prim__t_eff))
u_P_sec__t_eff = np.reshape(u_P_sec__t_eff, len(u_P_sec__t_eff))
u_P_inclination = np.reshape(u_P_inclination, len(u_P_inclination))
u_P_mass_ratio = np.reshape(u_P_mass_ratio, len(u_P_mass_ratio))
u_P_prim__surface_potential = np.reshape(u_P_prim__surface_potential, len(u_P_prim__surface_potential))
u_P_sec__surface_potential = np.reshape(u_P_sec__surface_potential, len(u_P_sec__surface_potential))
u_P_t1_t2 = np.reshape(u_P_t1_t2, len(u_P_t1_t2))
u_P_critical_surface_potential = np.reshape(u_P_critical_surface_potential, len(u_P_critical_surface_potential))
u_P_primary_equivalent_radius = np.reshape(u_P_primary_equivalent_radius, len(u_P_primary_equivalent_radius))
u_P_secondary_equivalent_radius = np.reshape(u_P_secondary_equivalent_radius, len(u_P_secondary_equivalent_radius))
u_P_primary_filling_factor = np.reshape(u_P_primary_filling_factor, len(u_P_primary_filling_factor))
u_P_secondary_filling_factor = np.reshape(u_P_secondary_filling_factor, len(u_P_secondary_filling_factor))


In [32]:
u_df = pd.DataFrame(data =
    {
    'primary__t_eff': [i[0] for i in unscaled_test],
    'P_prim__t_eff': u_P_prim__t_eff,
    'prim_t_eff_abs_diff': abs([i[0] for i in unscaled_test] - u_P_prim__t_eff),

    'secondary__t_eff': [i[1] for i in unscaled_test],
    'P_sec__t_eff': u_P_sec__t_eff,
    'sec_t_eff_abs_diff': abs([i[1] for i in unscaled_test] - u_P_sec__t_eff),

    'inclination': [i[2] for i in unscaled_test],
    'P_inclination': u_P_inclination,
    'inclination_abs_diff': abs([i[2] for i in unscaled_test] - u_P_inclination),

    'mass_ratio': [i[3] for i in unscaled_test],
    'P_mass_ratio': u_P_mass_ratio,
    'mass_ratio_abs_diff': abs([i[3] for i in unscaled_test] - u_P_mass_ratio),

    'prim_surface_potential': [i[4] for i in unscaled_test],
    'P_prim_surf_potential': u_P_prim__surface_potential,
    'prim_surf_potent_abs_diff': abs([i[4] for i in unscaled_test] - u_P_prim__surface_potential),

    'sec_surface_potential': [i[5] for i in unscaled_test],
    'P_sec_surf_potential': u_P_sec__surface_potential,
    'sec_surf_potent_abs_diff': abs([i[5] for i in unscaled_test] - u_P_sec__surface_potential),

    't1_t2': [i[6] for i in unscaled_test],
    'P_t1_t2': u_P_t1_t2,
    't1_t2_abs_diff': abs([i[6] for i in unscaled_test] - u_P_t1_t2),

    'critical_surf_potential': [i[7] for i in unscaled_test],
    'P_crit_surf_potential': u_P_critical_surface_potential,
    'crit_surf_potential_abs_diff': abs([i[7] for i in unscaled_test] - u_P_critical_surface_potential),

    'prim_equi_radius': [i[8] for i in unscaled_test],
    'P_prim_equi_radius': u_P_primary_equivalent_radius,
    'prim_equi_radius_abs_diff': abs([i[8] for i in unscaled_test] - u_P_primary_equivalent_radius),

    'sec_equi_radius': [i[9] for i in unscaled_test],
    'P_sec_equi_radius': u_P_secondary_equivalent_radius,
    'sec_equi_radius_abs_diff': abs([i[9] for i in unscaled_test] - u_P_secondary_equivalent_radius),

    'prim_filling_factor': [i[10] for i in unscaled_test],
    'P_prim_filling_factor': u_P_primary_filling_factor,
    'prim_filling_factor_abs_diff': abs([i[10] for i in unscaled_test] - u_P_primary_filling_factor),

    'sec_filling_factor': [i[11] for i in unscaled_test],
    'P_sec_filling_factor': u_P_secondary_filling_factor,
    'sec_filling_factor_abs_diff': abs([i[11] for i in unscaled_test] - u_P_secondary_filling_factor)
    })


In [33]:
u_df.head()

,primary__t_eff,P_prim__t_eff,prim_t_eff_abs_diff,secondary__t_eff,P_sec__t_eff,sec_t_eff_abs_diff,inclination,P_inclination,inclination_abs_diff,mass_ratio,...,prim_equi_radius_abs_diff,sec_equi_radius,P_sec_equi_radius,sec_equi_radius_abs_diff,prim_filling_factor,P_prim_filling_factor,prim_filling_factor_abs_diff,sec_filling_factor,P_sec_filling_factor,sec_filling_factor_abs_diff
0,35000.0,22590.541016,12409.458984,30000.0,16898.136719,13101.863281,1.397338,1.405728,0.008390,2.5,...,0.085097,0.249770,0.166328,0.083443,-12.574528,-10.491901,2.082626,-7.187790,-10.590524,3.402734
1,6000.0,21700.619141,15700.619141,5000.0,10727.498047,5727.498047,1.334076,1.425428,0.091351,10.0,...,0.043254,0.209660,0.080666,0.128994,-22.832011,-49.075897,26.243886,-44.789259,-54.459393,9.670133
2,18000.0,21157.845703,3157.845703,10000.0,12986.236328,2986.236328,1.456709,1.448875,0.007833,0.9,...,0.062401,0.210329,0.247129,0.036800,-0.809594,-6.498887,5.689293,-3.558689,-6.720229,3.161540
3,35000.0,25142.878906,9857.121094,10000.0,8668.715820,1331.284180,1.437048,1.414096,0.022952,0.8,...,0.074698,0.250978,0.308348,0.057371,-0.353527,0.165631,0.519157,-2.019105,-2.487958,0.468854
4,35000.0,27920.226562,7079.773438,8000.0,6926.838867,1073.161133,1.415526,1.414034,0.001492,1.0,...,0.043711,0.089977,0.131865,0.041888,-9.114192,-14.563114,5.448923,-15.399719,-8.027138,7.372581


In [34]:
# Dictionary with averages of absolute differencies between predicted and true data
u_avg_diff_dict = {
    "avg_primary_t_eff": (sum(u_df.prim_t_eff_abs_diff)/len(u_df.prim_t_eff_abs_diff)),
    "avg_secondary_prim_t_eff": (sum(u_df.sec_t_eff_abs_diff)/len(u_df.sec_t_eff_abs_diff)),
    "avg_inclination": (sum(u_df.inclination_abs_diff)/len(u_df.inclination_abs_diff)),
    "avg_mass_ratio": (sum(u_df.mass_ratio_abs_diff)/len(u_df.mass_ratio_abs_diff)),
    "avg_primary_surface_potential":(sum(u_df.prim_surf_potent_abs_diff)/len(u_df.prim_surf_potent_abs_diff)),
    "avg_secondary_surface_potential": (sum(u_df.sec_surf_potent_abs_diff)/len(u_df.sec_surf_potent_abs_diff)),
    "avg_t1_t2":(sum(u_df.t1_t2_abs_diff)/len(u_df.t1_t2_abs_diff)),
    "avg_crit_surf_potent":(sum(u_df.crit_surf_potential_abs_diff)/len(u_df.crit_surf_potential_abs_diff)),
    "avg_prim_equi_radius":(sum(u_df.prim_equi_radius_abs_diff)/len(u_df.prim_equi_radius_abs_diff)),
    "avg_sec_equi_radius":(sum(u_df.sec_equi_radius_abs_diff)/len(u_df.sec_equi_radius_abs_diff)),
    "avg_prim_fill_factor":(sum(u_df.prim_filling_factor_abs_diff)/len(u_df.prim_filling_factor_abs_diff)),
    "avg_sec_fill_factor":(sum(u_df.sec_filling_factor_abs_diff)/len(u_df.sec_filling_factor_abs_diff))
}

In [36]:
#UNSCALED PREDICTED AND TEST VALUES
pd.DataFrame(u_avg_diff_dict.items(), columns = ['Attribute', 'Unscaled Avg diff'])

,Attribute,Unscaled Avg diff
0,avg_primary_t_eff,7400.100594
1,avg_secondary_prim_t_eff,3538.858494
2,avg_inclination,0.051315
3,avg_mass_ratio,1.046665
4,avg_primary_surface_potential,5.704261
5,avg_secondary_surface_potential,15.127174
6,avg_t1_t2,0.645714
7,avg_crit_surf_potent,1.418464
8,avg_prim_equi_radius,0.030366
9,avg_sec_equi_radius,0.035939


## Prediction on normalized test data with noise

### working with normalized predicted and test data

In [39]:
y_pred_norm_n = model.predict(X_test_n)

In [40]:
P_prim__t_eff = [i[0] for i in y_pred_norm_n]
P_sec__t_eff = [i[1] for i in y_pred_norm_n]
P_inclination = [i[2] for i in y_pred_norm_n]
P_mass_ratio = [i[3] for i in y_pred_norm_n]
P_prim__surface_potential = [i[4] for i in y_pred_norm_n]
P_sec__surface_potential = [i[5] for i in y_pred_norm_n]
P_t1_t2 = [i[6] for i in y_pred_norm_n]
P_critical_surface_potential = [i[7] for i in y_pred_norm_n]
P_primary_equivalent_radius = [i[8] for i in y_pred_norm_n]
P_secondary_equivalent_radius = [i[9] for i in y_pred_norm_n]
P_primary_filling_factor = [i[10] for i in y_pred_norm_n]
P_secondary_filling_factor = [i[11] for i in y_pred_norm_n]

In [41]:
P_prim__t_eff = np.reshape(P_prim__t_eff, len(P_prim__t_eff))
P_sec__t_eff = np.reshape(P_sec__t_eff, len(P_sec__t_eff))
P_inclination = np.reshape(P_inclination, len(P_inclination))
P_mass_ratio = np.reshape(P_mass_ratio, len(P_mass_ratio))
P_prim__surface_potential = np.reshape(P_prim__surface_potential, len(P_prim__surface_potential))
P_sec__surface_potential = np.reshape(P_sec__surface_potential, len(P_sec__surface_potential))
P_t1_t2 = np.reshape(P_t1_t2, len(P_t1_t2))
P_critical_surface_potential = np.reshape(P_critical_surface_potential, len(P_critical_surface_potential))
P_primary_equivalent_radius = np.reshape(P_primary_equivalent_radius, len(P_primary_equivalent_radius))
P_secondary_equivalent_radius = np.reshape(P_secondary_equivalent_radius, len(P_secondary_equivalent_radius))
P_primary_filling_factor = np.reshape(P_primary_filling_factor, len(P_primary_filling_factor))
P_secondary_filling_factor = np.reshape(P_secondary_filling_factor, len(P_secondary_filling_factor))


In [42]:
df = pd.DataFrame(data =
    {
    'primary__t_eff': [i[0] for i in y_test_norm_n],
    'P_prim__t_eff': P_prim__t_eff,
    'prim_t_eff_abs_diff': abs([i[0] for i in y_test_norm_n] - P_prim__t_eff),

    'secondary__t_eff': [i[1] for i in y_test_norm_n],
    'P_sec__t_eff': P_sec__t_eff,
    'sec_t_eff_abs_diff': abs([i[1] for i in y_test_norm_n] - P_sec__t_eff),

    'inclination': [i[2] for i in y_test_norm_n],
    'P_inclination': P_inclination,
    'inclination_abs_diff': abs([i[2] for i in y_test_norm_n] - P_inclination),

    'mass_ratio': [i[3] for i in y_test_norm_n],
    'P_mass_ratio': P_mass_ratio,
    'mass_ratio_abs_diff': abs([i[3] for i in y_test_norm_n] - P_mass_ratio),

    'prim_surface_potential': [i[4] for i in y_test_norm_n],
    'P_prim_surf_potential': P_prim__surface_potential,
    'prim_surf_potent_abs_diff': abs([i[4] for i in y_test_norm_n] - P_prim__surface_potential),

    'sec_surface_potential': [i[5] for i in y_test_norm_n],
    'P_sec_surf_potential': P_sec__surface_potential,
    'sec_surf_potent_abs_diff': abs([i[5] for i in y_test_norm_n] - P_sec__surface_potential),

    't1_t2': [i[6] for i in y_test_norm_n],
    'P_t1_t2': P_t1_t2,
    't1_t2_abs_diff': abs([i[6] for i in y_test_norm_n] - P_t1_t2),

    'critical_surf_potential': [i[7] for i in y_test_norm_n],
    'P_crit_surf_potential': P_critical_surface_potential,
    'crit_surf_potential_abs_diff': abs([i[7] for i in y_test_norm_n] - P_critical_surface_potential),

    'prim_equi_radius': [i[8] for i in y_test_norm_n],
    'P_prim_equi_radius': P_primary_equivalent_radius,
    'prim_equi_radius_abs_diff': abs([i[8] for i in y_test_norm_n] - P_primary_equivalent_radius),

    'sec_equi_radius': [i[9] for i in y_test_norm_n],
    'P_sec_equi_radius': P_secondary_equivalent_radius,
    'sec_equi_radius_abs_diff': abs([i[9] for i in y_test_norm_n] - P_secondary_equivalent_radius),

    'prim_filling_factor': [i[10] for i in y_test_norm_n],
    'P_prim_filling_factor': P_primary_filling_factor,
    'prim_filling_factor_abs_diff': abs([i[10] for i in y_test_norm_n] - P_primary_filling_factor),

    'sec_filling_factor': [i[11] for i in y_test_norm_n],
    'P_sec_filling_factor': P_secondary_filling_factor,
    'sec_filling_factor_abs_diff': abs([i[11] for i in y_test_norm_n] - P_secondary_filling_factor)
    })


In [43]:
df.head()

,primary__t_eff,P_prim__t_eff,prim_t_eff_abs_diff,secondary__t_eff,P_sec__t_eff,sec_t_eff_abs_diff,inclination,P_inclination,inclination_abs_diff,mass_ratio,...,prim_equi_radius_abs_diff,sec_equi_radius,P_sec_equi_radius,sec_equi_radius_abs_diff,prim_filling_factor,P_prim_filling_factor,prim_filling_factor_abs_diff,sec_filling_factor,P_sec_filling_factor,sec_filling_factor_abs_diff
0,0.756098,0.441306,0.314791,0.634146,0.306674,0.327472,0.791784,0.789012,0.002772,0.242424,...,0.126177,0.438463,0.335403,0.103061,0.991633,0.996302,0.004669,0.995217,0.996379,0.001162
1,0.756098,0.453344,0.302754,0.634146,0.314675,0.319472,0.791784,0.801761,0.009977,0.242424,...,0.155459,0.438463,0.286260,0.152203,0.991633,0.993028,0.001395,0.995217,0.993006,0.002211
2,0.756098,0.444294,0.311803,0.634146,0.319200,0.314946,0.791784,0.824460,0.032676,0.242424,...,0.141212,0.438463,0.298945,0.139518,0.991633,0.995206,0.003573,0.995217,0.991253,0.003964
3,0.048780,0.388199,0.339419,0.024390,0.129081,0.104691,0.715846,0.837243,0.121397,1.000000,...,0.101193,0.365116,0.093909,0.271207,0.984807,0.969154,0.015653,0.970197,0.962420,0.007776
4,0.048780,0.404118,0.355337,0.024390,0.180031,0.155641,0.715846,0.770481,0.054634,1.000000,...,0.115509,0.365116,0.136260,0.228856,0.984807,0.979412,0.005396,0.970197,0.975196,0.005000


In [ ]:
# df.to_csv('all_params_predictions/DetachedSynthetic_normalized_allParams_OneModel.csv', index=False)

In [44]:
# Dictionary with averages of absolute differencies between predicted and true data
avg_diff_dict_n = {
    "avg_primary_t_eff": (sum(df.prim_t_eff_abs_diff)/len(df.prim_t_eff_abs_diff)),
    "avg_secondary_prim_t_eff": (sum(df.sec_t_eff_abs_diff)/len(df.sec_t_eff_abs_diff)),
    "avg_inclination": (sum(df.inclination_abs_diff)/len(df.inclination_abs_diff)),
    "avg_mass_ratio": (sum(df.mass_ratio_abs_diff)/len(df.mass_ratio_abs_diff)),
    "avg_primary_surface_potential":(sum(df.prim_surf_potent_abs_diff)/len(df.prim_surf_potent_abs_diff)),
    "avg_secondary_surface_potential": (sum(df.sec_surf_potent_abs_diff)/len(df.sec_surf_potent_abs_diff)),
    "avg_t1_t2":(sum(df.t1_t2_abs_diff)/len(df.t1_t2_abs_diff)),
    "avg_crit_surf_potent":(sum(df.crit_surf_potential_abs_diff)/len(df.crit_surf_potential_abs_diff)),
    "avg_prim_equi_radius":(sum(df.prim_equi_radius_abs_diff)/len(df.prim_equi_radius_abs_diff)),
    "avg_sec_equi_radius":(sum(df.sec_equi_radius_abs_diff)/len(df.sec_equi_radius_abs_diff)),
    "avg_prim_fill_factor":(sum(df.prim_filling_factor_abs_diff)/len(df.prim_filling_factor_abs_diff)),
    "avg_sec_fill_factor":(sum(df.sec_filling_factor_abs_diff)/len(df.sec_filling_factor_abs_diff))
}

In [45]:
pd.DataFrame(avg_diff_dict_n.items(), columns = ['Attribute', 'Avg diff'])

,Attribute,Avg diff
0,avg_primary_t_eff,0.181298
1,avg_secondary_prim_t_eff,0.086934
2,avg_inclination,0.063106
3,avg_mass_ratio,0.107505
4,avg_primary_surface_potential,0.055881
5,avg_secondary_surface_potential,0.015267
6,avg_t1_t2,0.063777
7,avg_crit_surf_potent,0.109937
8,avg_prim_equi_radius,0.058338
9,avg_sec_equi_radius,0.068304


## Normalized predicted values and test values with noise to original

In [37]:
unscaled_test_n = scaler.inverse_transform(y_test_norm_n)
unscaled_test_n[0]

array([ 3.50000000e+04,  3.00000000e+04,  1.39733788e+00,  2.50000000e+00,
        1.36152222e+01,  1.03295175e+01,  1.16666667e+00,  5.94523973e+00,
        9.00097983e-02,  2.49770299e-01, -1.25745279e+01, -7.18778997e+00])

In [46]:
unscaled_pred_n = scaler.inverse_transform(y_pred_norm_n)
unscaled_pred_n[0]

array([ 2.2093564e+04,  1.6573641e+04,  1.3950281e+00,  1.8420665e+00,
        8.4321756e+00,  1.0671171e+01,  1.0560663e+00,  4.7761393e+00,
        1.5900953e-01,  1.9341147e-01, -5.5581641e+00, -5.4421635e+00],
      dtype=float32)

In [47]:
un_P_prim__t_eff = [i[0] for i in unscaled_pred_n]
un_P_sec__t_eff = [i[1] for i in unscaled_pred_n]
un_P_inclination = [i[2] for i in unscaled_pred_n]
un_P_mass_ratio = [i[3] for i in unscaled_pred_n]
un_P_prim__surface_potential = [i[4] for i in unscaled_pred_n]
un_P_sec__surface_potential = [i[5] for i in unscaled_pred_n]
un_P_t1_t2 = [i[6] for i in unscaled_pred_n]
un_P_critical_surface_potential = [i[7] for i in unscaled_pred_n]
un_P_primary_equivalent_radius = [i[8] for i in unscaled_pred_n]
un_P_secondary_equivalent_radius = [i[9] for i in unscaled_pred_n]
un_P_primary_filling_factor = [i[10] for i in unscaled_pred_n]
un_P_secondary_filling_factor = [i[11] for i in unscaled_pred_n]

In [48]:
un_P_prim__t_eff = np.reshape(un_P_prim__t_eff, len(un_P_prim__t_eff))
un_P_sec__t_eff = np.reshape(un_P_sec__t_eff, len(un_P_sec__t_eff))
un_P_inclination = np.reshape(un_P_inclination, len(un_P_inclination))
un_P_mass_ratio = np.reshape(un_P_mass_ratio, len(un_P_mass_ratio))
un_P_prim__surface_potential = np.reshape(un_P_prim__surface_potential, len(un_P_prim__surface_potential))
un_P_sec__surface_potential = np.reshape(un_P_sec__surface_potential, len(un_P_sec__surface_potential))
un_P_t1_t2 = np.reshape(un_P_t1_t2, len(un_P_t1_t2))
un_P_critical_surface_potential = np.reshape(un_P_critical_surface_potential, len(un_P_critical_surface_potential))
un_P_primary_equivalent_radius = np.reshape(un_P_primary_equivalent_radius, len(un_P_primary_equivalent_radius))
un_P_secondary_equivalent_radius = np.reshape(un_P_secondary_equivalent_radius, len(un_P_secondary_equivalent_radius))
un_P_primary_filling_factor = np.reshape(un_P_primary_filling_factor, len(un_P_primary_filling_factor))
un_P_secondary_filling_factor = np.reshape(un_P_secondary_filling_factor, len(un_P_secondary_filling_factor))


In [50]:
un_df = pd.DataFrame(data =
    {
    'primary__t_eff': [i[0] for i in unscaled_test_n],
    'P_prim__t_eff': un_P_prim__t_eff,
    'prim_t_eff_abs_diff': abs([i[0] for i in unscaled_test_n] - un_P_prim__t_eff),

    'secondary__t_eff': [i[1] for i in unscaled_test_n],
    'P_sec__t_eff': un_P_sec__t_eff,
    'sec_t_eff_abs_diff': abs([i[1] for i in unscaled_test_n] - un_P_sec__t_eff),

    'inclination': [i[2] for i in unscaled_test_n],
    'P_inclination': un_P_inclination,
    'inclination_abs_diff': abs([i[2] for i in unscaled_test_n] - un_P_inclination),

    'mass_ratio': [i[3] for i in unscaled_test_n],
    'P_mass_ratio': un_P_mass_ratio,
    'mass_ratio_abs_diff': abs([i[3] for i in unscaled_test_n] - un_P_mass_ratio),

    'prim_surface_potential': [i[4] for i in unscaled_test_n],
    'P_prim_surf_potential': un_P_prim__surface_potential,
    'prim_surf_potent_abs_diff': abs([i[4] for i in unscaled_test_n] - un_P_prim__surface_potential),

    'sec_surface_potential': [i[5] for i in unscaled_test_n],
    'P_sec_surf_potential': un_P_sec__surface_potential,
    'sec_surf_potent_abs_diff': abs([i[5] for i in unscaled_test_n] - un_P_sec__surface_potential),

    't1_t2': [i[6] for i in unscaled_test_n],
    'P_t1_t2': un_P_t1_t2,
    't1_t2_abs_diff': abs([i[6] for i in unscaled_test_n] - un_P_t1_t2),

    'critical_surf_potential': [i[7] for i in unscaled_test_n],
    'P_crit_surf_potential': un_P_critical_surface_potential,
    'crit_surf_potential_abs_diff': abs([i[7] for i in unscaled_test_n] - un_P_critical_surface_potential),

    'prim_equi_radius': [i[8] for i in unscaled_test_n],
    'P_prim_equi_radius': un_P_primary_equivalent_radius,
    'prim_equi_radius_abs_diff': abs([i[8] for i in unscaled_test_n] - un_P_primary_equivalent_radius),

    'sec_equi_radius': [i[9] for i in unscaled_test_n],
    'P_sec_equi_radius': un_P_secondary_equivalent_radius,
    'sec_equi_radius_abs_diff': abs([i[9] for i in unscaled_test_n] - un_P_secondary_equivalent_radius),

    'prim_filling_factor': [i[10] for i in unscaled_test_n],
    'P_prim_filling_factor': un_P_primary_filling_factor,
    'prim_filling_factor_abs_diff': abs([i[10] for i in unscaled_test_n] - un_P_primary_filling_factor),

    'sec_filling_factor': [i[11] for i in unscaled_test_n],
    'P_sec_filling_factor': un_P_secondary_filling_factor,
    'sec_filling_factor_abs_diff': abs([i[11] for i in unscaled_test_n] - un_P_secondary_filling_factor)
    })


In [51]:
un_df.head()

,primary__t_eff,P_prim__t_eff,prim_t_eff_abs_diff,secondary__t_eff,P_sec__t_eff,sec_t_eff_abs_diff,inclination,P_inclination,inclination_abs_diff,mass_ratio,...,prim_equi_radius_abs_diff,sec_equi_radius,P_sec_equi_radius,sec_equi_radius_abs_diff,prim_filling_factor,P_prim_filling_factor,prim_filling_factor_abs_diff,sec_filling_factor,P_sec_filling_factor,sec_filling_factor_abs_diff
0,35000.0,22093.564453,12906.435547,30000.0,16573.640625,13426.359375,1.397338,1.395028,0.002310,2.5,...,0.069000,0.24977,0.193411,0.056359,-12.574528,-5.558164,7.016364,-7.187790,-5.442163,1.745627
1,35000.0,22587.095703,12412.904297,30000.0,16901.662109,13098.337891,1.397338,1.405649,0.008312,2.5,...,0.085013,0.24977,0.166538,0.083233,-12.574528,-10.477480,2.097048,-7.187790,-10.510444,3.322654
2,35000.0,22216.058594,12783.941406,30000.0,17087.203125,12912.796875,1.397338,1.424559,0.027221,2.5,...,0.077222,0.24977,0.173475,0.076296,-12.574528,-7.205281,5.369247,-7.187790,-13.144953,5.957163
3,6000.0,19916.175781,13916.175781,5000.0,9292.312500,4292.312500,1.334076,1.435209,0.101132,10.0,...,0.055337,0.20966,0.061350,0.148310,-22.832011,-46.356380,23.524369,-44.789259,-56.475922,11.686662
4,6000.0,20568.830078,14568.830078,5000.0,11381.290039,6381.290039,1.334076,1.379591,0.045514,10.0,...,0.063166,0.20966,0.084510,0.125150,-22.832011,-30.941217,8.109206,-44.789259,-37.275738,7.513522


In [52]:
# Dictionary with averages of absolute differencies between predicted and true data
un_avg_diff_dict = {
    "avg_primary_t_eff": (sum(un_df.prim_t_eff_abs_diff)/len(un_df.prim_t_eff_abs_diff)),
    "avg_secondary_prim_t_eff": (sum(un_df.sec_t_eff_abs_diff)/len(un_df.sec_t_eff_abs_diff)),
    "avg_inclination": (sum(un_df.inclination_abs_diff)/len(un_df.inclination_abs_diff)),
    "avg_mass_ratio": (sum(un_df.mass_ratio_abs_diff)/len(un_df.mass_ratio_abs_diff)),
    "avg_primary_surface_potential":(sum(un_df.prim_surf_potent_abs_diff)/len(un_df.prim_surf_potent_abs_diff)),
    "avg_secondary_surface_potential": (sum(un_df.sec_surf_potent_abs_diff)/len(un_df.sec_surf_potent_abs_diff)),
    "avg_t1_t2":(sum(un_df.t1_t2_abs_diff)/len(un_df.t1_t2_abs_diff)),
    "avg_crit_surf_potent":(sum(un_df.crit_surf_potential_abs_diff)/len(un_df.crit_surf_potential_abs_diff)),
    "avg_prim_equi_radius":(sum(un_df.prim_equi_radius_abs_diff)/len(un_df.prim_equi_radius_abs_diff)),
    "avg_sec_equi_radius":(sum(un_df.sec_equi_radius_abs_diff)/len(un_df.sec_equi_radius_abs_diff)),
    "avg_prim_fill_factor":(sum(un_df.prim_filling_factor_abs_diff)/len(un_df.prim_filling_factor_abs_diff)),
    "avg_sec_fill_factor":(sum(un_df.sec_filling_factor_abs_diff)/len(un_df.sec_filling_factor_abs_diff))
}

In [53]:
#UNSCALED PREDICTED AND TEST VALUES
pd.DataFrame(un_avg_diff_dict.items(), columns = ['Attribute', 'Unscaled Avg diff'])

,Attribute,Unscaled Avg diff
0,avg_primary_t_eff,7433.216833
1,avg_secondary_prim_t_eff,3564.278638
2,avg_inclination,0.052572
3,avg_mass_ratio,1.064302
4,avg_primary_surface_potential,6.034011
5,avg_secondary_surface_potential,15.182369
6,avg_t1_t2,0.653716
7,avg_crit_surf_potent,1.443692
8,avg_prim_equi_radius,0.031902
9,avg_sec_equi_radius,0.037352
